In [ ]:
import sys
sys.stdout = open("/dev/stdout", "w")
print("Now my output shall be on terminal, not on my notebook")

In [ ]:
# here we implement windowed candle data for 1 minute
# here we implement windowed candle data for 3 minute
# here we implement windowed candle data for 5 minute

# we use hadoop-vm as hostname, which is running on another system


#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic stock-ticks 
    
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-1min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-1min  --from-beginning
 
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-3min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-3min  --from-beginning

#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-5min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-5min  --from-beginning

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles3min").getOrCreate()

In [ ]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1
# GROUP ID SHOULD BE CHANGED FOR EVERY PERSON 
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "hadoop-vm:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group234")\
  .load()

In [ ]:
# key and value are binary type, we need to CAST To STRING type
kafkaDf.printSchema()
# timestampType values
# CreateTime:  Timestamp relates to message creation time as set by a Kafka client/producer
# LogAppendTime : Timestamp relates to the time a message was appended to a Kafka log.
# UnknownType

In [ ]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, timestamp as we need for 10 minute window
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema() # we get only value as string

In [ ]:
# echoOnconsole = ticksDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

schema = StructType( [
    StructField("symbol", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("volume", LongType(), True),
    StructField("timestamp", LongType(),  True)
])

In [ ]:
# we parse value which is JSON string to kafka struct, 
# this create a new column [replacing existing value which is string] with type
jsonDf = ticksDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

In [ ]:
# now extract all the sub fields symbol, price, volume, timestamp into data frame
# exisitng value, timestamp shall be removed. existing timestamp is value produced time, not a stock tick time
# value.timestamp that comes from producer is basically a event time
stockTickDf = jsonDf.select (F.col("value.*")) # extract all sub fields as schema
stockTickDf.printSchema()

In [ ]:
# we have data coming in stream, each record has timestamp
# we have milli seconds 1647283268253, the actual seconds Mon Mar 14 2022 14:41:08
# now we have to calculate 1 minute window, sum(volume) up to 1 minute
# after conversions, we take timestamp which is truncatd to minute instead of milli seconds, drop temp columns
stockTickDf = stockTickDf\
                .withColumn("timestamp", (F.col("timestamp") / 1000).cast("timestamp"))\
                

stockTickDf.printSchema()
# echoOnconsole = stockTickDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [ ]:
# aggregate
# sum (volume), sum(traded_value), candles open, high, low, close [OHLC]
# groupBy by symbol, timestamp

# now add delayed record processing with waterMark
stockTickDf3Min = stockTickDf\
                            .selectExpr("timestamp", "symbol",  "struct(*) as value")\
                            .withWatermark("timestamp", "1 minutes")\
                            .groupBy("symbol", F.window("timestamp", "60 seconds"))\
                          .agg( F.collect_list("value").alias("ticks"))

stockTickDf3Min.printSchema()

In [ ]:
# spark stream with batch output
# spark basically process the data as batch
# for developer convinence, we have api, that can give us finite data frame for writing to file/jsbc/db/custom coding
# process discrete output batch
# this function is called for every update/delete/comlete triggers with result data set as dataframe
# candleBatchDf is BATCH DATAFRAME
# stockTickDf1Min is STREAM DATAFRAME
def processBatchData(candleBatchDf, batch_id):
    from pyspark.sql.functions import explode, col

    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
    #result = candleBatchDf.collect()
    #print(result)
    #candleBatchDf.printSchema()
    #candleBatchDf.show(2, truncate = False)
    
    # pick all the elements inside ticks array into element
    # breaking group using explode
    candleTickBatchDf = (candleBatchDf.select("window", explode(F.column("ticks")) )
                                      .select(F.col("window.*"), F.col("col.*"))
                        )
    
    candleTickBatchDf.printSchema()
    candleTickBatchDf.show(2, truncate=False)
    # write to csv/json/parqeut/database /jdbc etc
    # window is nested column, has its children columns called start and end
    # start and end are part of nested window column
    # append mode will not delete existin data, instead append to existing table
     
    
stockTickDf3Min.writeStream.outputMode("append").foreachBatch(processBatchData).start()
    
# now go to mysql, check candle_1min exist
# show tables;
# select * from candle_1min

In [ ]:
# write result to kafka
# we need to publish all the field together as value
# we take all columns, convert to json string
# publish json as value to kafka

stockTickDf3MinKafka = stockTickDf3Min\
                            .selectExpr("to_json(struct(*)) AS value")

stockTickDf3MinKafka\
            .writeStream\
             .format("kafka")\
            .outputMode("update")\
             .option("kafka.bootstrap.servers", "localhost:9092")\
            .option("topic", "candles-3min")\
            .option("checkpointLocation", "file:///tmp/spark5")\
            .start()